In [31]:
#import dependencies
import os
import pandas as pd
import glob

In [32]:
# #I want to display the data to better understand what I am working with
# #path = 'low_freq/house_1/'
# #forward slash to access path and a one more slash to access the folder of the file
# #houses 4 and 6 have air conditionong system
# path = 'E:/Document/Master/Dataset/Energy_disaggregation/REDD_dataset/low_freq/house_1/' 
# file = path + 'channel_1.dat'
# df = pd.read_table(file, sep= ' ')

In [58]:
def read_label():
    label = {}
    for i in range(1, 7):
        hi = 'E:/Document/Master/Dataset/Energy_disaggregation/REDD_dataset/low_freq/house_{}/labels.dat'.format(i)
        label[i] = {}
        with open(hi) as f:
            for line in f:
                splitted_line = line.split(' ')
                label[i][int(splitted_line[0])] = splitted_line[1].strip() + '_' + splitted_line[0]
    return label
labels = read_label()
for i in range(4,7,2): # the third variable is to step
    print('House {}: '.format(i), labels[i], '\n')
   

        

House 4:  {1: 'mains_1', 2: 'mains_2', 3: 'lighting_3', 4: 'furance_4', 5: 'kitchen_outlets_5', 6: 'outlets_unknown_6', 7: 'washer_dryer_7', 8: 'stove_8', 9: 'air_conditioning_9', 10: 'air_conditioning_10', 11: 'miscellaeneous_11', 12: 'smoke_alarms_12', 13: 'lighting_13', 14: 'kitchen_outlets_14', 15: 'dishwaser_15', 16: 'bathroom_gfi_16', 17: 'bathroom_gfi_17', 18: 'lighting_18', 19: 'lighting_19', 20: 'air_conditioning_20'} 

House 6:  {1: 'mains_1', 2: 'mains_2', 3: 'kitchen_outlets_3', 4: 'washer_dryer_4', 5: 'stove_5', 6: 'electronics_6', 7: 'bathroom_gfi_7', 8: 'refrigerator_8', 9: 'dishwaser_9', 10: 'outlets_unknown_10', 11: 'outlets_unknown_11', 12: 'electric_heat_12', 13: 'kitchen_outlets_13', 14: 'lighting_14', 15: 'air_conditioning_15', 16: 'air_conditioning_16', 17: 'air_conditioning_17'} 

